In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

# # To show the interactive widgets 
# %matplotlib widget
# from Pyriod import Pyriod #!This hates me. So just using LK
# import warnings

# # Suppress warning from Astropy module 
# # astropy.stats used in Pyriod is no longer supported
# from astropy.utils.exceptions import AstropyDeprecationWarning
# warnings.filterwarnings("ignore", category=AstropyDeprecationWarning)

# # Suppress invalid value runtime warning from Astropy and Pyriod 
# warnings.filterwarnings("ignore", category=RuntimeWarning) 

# from IPython.display import Image

# import lightkurve as lk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightkurve as lk
import astropy.units as u
%matplotlib widget



def lcPlots(fName):
    """
    Uses LightKurve to make LC plots from just a filename of a csv with flux and Time cols.
    """
    data = pd.read_csv(fName)
    fluxes = data["flux"]
    times = data["Time"]

    fluxErrs = np.ones_like(fluxes)*1e-4 #lk needs errors

    lc =lk.LightCurve(data=None, time=times, flux=fluxes, flux_err=fluxErrs)
    lc = lc.normalize()
    lc.scatter()
    pg = lc.to_periodogram()

    pg.plot(scale="log")
    period = pg.period_at_max_power
    print(period)
    lc.fold(period).scatter()

    lcModel = pg.model(time=lc.time, frequency=pg.frequency_at_max_power)
    ax = lc.plot()
    lcModel.plot(ax=ax, lw=3, ls='--',c='red')
    return lcModel


objName = " Ruff "


fName = f"./{objName}Matches.csv"
lcm = lcPlots(fName=fName)



In [ ]:
lcm.plot()
vars(lcm)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib widget

from astropy.timeseries import LombScargle as lsp


fName = f"./{objName}Matches.csv"
data = pd.read_csv(fName)
fluxes = data["flux"]
times = data["Time"]-2400000.5


freq, powers = lsp(times*u.day, fluxes).autopower(samples_per_peak=50)
bestPow = np.max(powers)
bestFreq = freq[np.argmax(powers)]
# print(bestFreq)

t_fit=np.linspace(times.min(),times.max(),1000)
ls = lsp(times, fluxes)
y_fit = ls.model(t_fit, bestFreq)

fig, ax = plt.subplots(2, figsize = (6,8))

ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set(xlabel="Log Frequency [1/d]", ylabel="Power")
ax[0].plot(freq, powers, c="k", label="Periodogram")
ax[0].scatter(bestFreq,bestPow, c="gold",marker="*", s=100, label=f"Best Frequency = {bestFreq.round(2)} /d")
ax[0].legend()

ax[1].set(xlabel="Time [MJD]", ylabel="Flux")
ax[1].scatter(times, fluxes, label = "Light curve", c="tab:orange", marker = "d", s=10)
ax[1].plot(t_fit,y_fit, ls="--", c="k", label="Model")




ls.false_alarm_probability(np.max(powers))

